In [1]:
import pandas as pd
import numpy as np
import cv2
import json
import os
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt


In [2]:
def create_binary_masks_from_via_csv(csv_path, images_dir, output_dir):
    """
    Create binary masks from VIA CSV annotations.
    
    Args:
        csv_path: Path to the VIA CSV annotation file
        images_dir: Directory containing the original images
        output_dir: Directory where mask images will be saved
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Read the CSV file
    via_df = pd.read_csv(csv_path)
    
    # Group by filename to process each image
    grouped = via_df.groupby('filename')
    
    for filename, group in grouped:
        # Get the original image to determine dimensions
        img_path = os.path.join(images_dir, filename)
        try:
            img = cv2.imread(img_path)
            if img is None:
                print(f"Warning: Could not read image {img_path}")
                continue
                
            height, width = img.shape[:2]
            
            # Create an empty mask with the same dimensions as the original image
            mask = np.zeros((height, width), dtype=np.uint8)
            
            # Process each annotation for this image
            for _, row in group.iterrows():
                # Parse the region attributes from the CSV
                region_data = json.loads(row['region_shape_attributes'])
                
                # Check if it's a polygon annotation
                if 'all_points_x' in region_data and 'all_points_y' in region_data:
                    # Get the polygon points
                    points_x = region_data['all_points_x']
                    points_y = region_data['all_points_y']
                    
                    # Combine x and y points into an array of points
                    points = np.array(list(zip(points_x, points_y)), dtype=np.int32)
                    
                    # Draw the filled polygon on the mask
                    cv2.fillPoly(mask, [points], 255)
                
                # Handle other shape types if needed (rectangles, circles, etc.)
                elif 'x' in region_data and 'width' in region_data:
                    # Rectangle annotation
                    x = region_data['x']
                    y = region_data['y']
                    w = region_data['width']
                    h = region_data['height']
                    cv2.rectangle(mask, (x, y), (x+w, y+h), 255, -1)  # -1 means filled
            
            # Save the mask
            mask_filename = os.path.splitext(filename)[0] + "_mask.png"
            mask_path = os.path.join(output_dir, mask_filename)
            cv2.imwrite(mask_path, mask)
            
            print(f"Created mask for {filename} at {mask_path}")
            
        except Exception as e:
            print(f"Error processing {filename}: {str(e)}")

In [3]:
def create_masks_from_via_json(json_path, images_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    
    with open(json_path, 'r') as f:
        via_data = json.load(f)
    
    for image_id, image_data in via_data.items():
        filename = image_data['filename']
        regions = image_data['regions']
        
        # Get image dimensions
        img_path = os.path.join(images_dir, filename)
        img = cv2.imread(img_path)
        if img is None:
            continue
        
        height, width = img.shape[:2]
        mask = np.zeros((height, width), dtype=np.uint8)
        
        for region in regions:
            shape_attributes = region['shape_attributes']
            
            if shape_attributes['name'] == 'polygon':
                points_x = shape_attributes['all_points_x']
                points_y = shape_attributes['all_points_y']
                points = np.array(list(zip(points_x, points_y)), dtype=np.int32)
                cv2.fillPoly(mask, [points], 255)
            
            # Handle other shapes as needed
        
        mask_filename = os.path.splitext(filename)[0] + "_mask.png"
        mask_path = os.path.join(output_dir, mask_filename)
        cv2.imwrite(mask_path, mask)